In [41]:
from sklearn.feature_extraction import DictVectorizer #字典数据抽取
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer#文本特征抽取
from sklearn.preprocessing import MinMaxScaler# 归一化
from sklearn.preprocessing import StandardScaler# 标准化
from sklearn.feature_selection import VarianceThreshold#特征选择
from sklearn.decomposition import PCA#主成分分析
import jieba#中文分词
import numpy as np#科学计算
from sklearn.impute import SimpleImputer#缺失值处理

# 1 数据中含有字符串的时候，如何做特征抽取

In [42]:
def dict2vec(): #字典数据抽取
#NO1 实例化，DictVectorizer()
#NO2 sparse改为True,输出的是每个不为零位置的坐标
# （矩阵中存在大量的0，sparse存储只记录非零位置，节省空间的作用），
# 稀疏矩阵可以节省存储空间
#NO3 给一个字典实例 #每个样本都是一个字典，有三个样本
#NO4 调用fit_transform
#NO5 打印结果
# vectorizer中文含义是矢量器的含义

    dict = DictVectorizer(sparse=True)  # 把sparse改为True看看


    # 调用fit_transform
    data = dict.fit_transform([{'city': '北京', 'temperature': 100},
                               {'city': '上海', 'temperature': 60},
                               {'city': '深圳', 'temperature': 30}])
    print(data)
    print('-' * 50)
    # 字典中的一些类别数据，分别进行转换成特征
    print(dict.get_feature_names_out())
    print('-' * 50)
    #去看每个特征代表的含义，逆转回去
    print(dict.inverse_transform(data))
    return None

dict2vec()

  (0, 1)	1.0
  (0, 3)	100.0
  (1, 0)	1.0
  (1, 3)	60.0
  (2, 2)	1.0
  (2, 3)	30.0
--------------------------------------------------
['city=上海' 'city=北京' 'city=深圳' 'temperature']
--------------------------------------------------
[{'city=北京': 1.0, 'temperature': 100.0}, {'city=上海': 1.0, 'temperature': 60.0}, {'city=深圳': 1.0, 'temperature': 30.0}]


# 2 一段英文文本如何变为数值类型

In [43]:
#text---->number(vector)
def text2vec_en():

#NO1
# 实例化CountVectorizer
# 默认会去除单个字母的单词，默认认为这个词对整个样本没有影响
# max_df, min_df--整数：
# 指每个词的所有文档词频数不小于min_df
# 出现该词的文档数目小于等于max_df
# max_df, min_df--小数：

# 某个词的出现的次数／所有文档数量
# min_df=2
    vector = CountVectorizer(min_df=1)
#NO2
# 调用fit_transform输入并转换数据
    test1="life is  short,i like python life"
    test2="life is too long,i dislike python"
    test3="life is short"
    res = vector.fit_transform([test1,test2,test3])

    # 打印结果,把每个词都分离了
    print(vector.get_feature_names_out())
    print('-'*50)
    print(res) #稀疏矩阵
    print('-'*50)
    print(type(res)) #<class 'scipy.sparse.csr.csr_matrix'>
    print('-'*50)
    print(res.toarray()) #转换成数组
    print('-'*50)
    # 逆转换，已经去除单个字母了
    print(vector.inverse_transform(res))


text2vec_en()

['dislike' 'is' 'life' 'like' 'long' 'python' 'short' 'too']
--------------------------------------------------
  (0, 2)	2
  (0, 1)	1
  (0, 6)	1
  (0, 3)	1
  (0, 5)	1
  (1, 2)	1
  (1, 1)	1
  (1, 5)	1
  (1, 7)	1
  (1, 4)	1
  (1, 0)	1
  (2, 2)	1
  (2, 1)	1
  (2, 6)	1
--------------------------------------------------
<class 'scipy.sparse._csr.csr_matrix'>
--------------------------------------------------
[[0 1 2 1 0 1 1 0]
 [1 1 1 0 1 1 0 1]
 [0 1 1 0 0 0 1 0]]
--------------------------------------------------
[array(['life', 'is', 'short', 'like', 'python'], dtype='<U7'), array(['life', 'is', 'python', 'too', 'long', 'dislike'], dtype='<U7'), array(['life', 'is', 'short'], dtype='<U7')]


# 3 一段汉字文本如何数值化，对于汉字不能用空格来分割

In [44]:
# chinese_text---->number(vector)
def text2vec_cn():
    """
    对文本进行特征值化,单个汉字单个字母不统计，因为单个汉字字母没有意义
    :return: None
    """
# NO1 实例化CountVectorizer
    cv = CountVectorizer()
# NO2 调用fit_transform输入并转换数据
    chinese_text1 = "人生苦短，我 喜欢 python python"
    chinese_text2 = "人生漫长，你们，不用， python，哼"
    data = cv.fit_transform([chinese_text1, chinese_text2])

    print(cv.get_feature_names_out())
    print('-' * 50)
    print(data)
    print('-' * 50)
    print(data.toarray())
    print('-' * 50)
    #单个汉字也去除了
    print(cv.inverse_transform(data))

    return None


text2vec_cn()


['python' '不用' '人生漫长' '人生苦短' '你们' '喜欢']
--------------------------------------------------
  (0, 3)	1
  (0, 5)	1
  (0, 0)	2
  (1, 0)	1
  (1, 2)	1
  (1, 4)	1
  (1, 1)	1
--------------------------------------------------
[[2 0 0 1 0 1]
 [1 1 1 0 1 0]]
--------------------------------------------------
[array(['人生苦短', '喜欢', 'python'], dtype='<U6'), array(['python', '人生漫长', '你们', '不用'], dtype='<U6')]


### 这段代码解释了中文text2vec_cn()中的原理和过程，以及分词的使用

In [45]:
#掌握如何对中文进行分词
def cutword():
    """
    通过jieba对中文进行分词
    :return:
    """
    originText1="今天很残酷，明天更残酷，后天很美好，但绝对大部分是死在明天晚上，所以每个人不要放弃今天。"

    originText2="我们看到的从很远星系来的光是在几百万年之前发出的，这样当我们看到宇宙时，我们是在看它的过去。"

    originText3="如果只用一种方式了解某样事物，你就不会真正了解它。了解事物真正含义的秘密取决于如何将其与我们所了解的事物相联系。"
    con1 = jieba.cut( originText1)

    con2 = jieba.cut( originText2)

    con3 = jieba.cut(originText3)

    # 转换成列表
    # print(type(con1))
    # print('-' * 50)
    content1 = list(con1)
    # print('-' * 50)
    content2 = list(con2)
    # print('-' * 50)
    content3 = list(con3)
    # print('-' * 50)
    # print(content1)
    # print('-' * 50)
    # print(content2)
    # print('-' * 50)
    # print(content3)
    # print('-' * 50)
    # 把列表转换成字符串
    c1 = ' '.join(content1)
    c2 = ' '.join(content2)
    c3 = ' '.join(content3)
    # print('-' * 50)
    # print(c1)
    # print('-' * 50)
    # print(c2)
    # print('-' * 50)
    # print(c3)
    # print('-' * 50)


    return c1, c2, c3


def hanzivec():
    """
    中文特征值化
    :return: None
    """
    c1, c2, c3 = cutword()
    print('hanzivec')
    print(c1)
    print('-'*50)
    print(c2)
    print('-'*50)
    print(c3)
    print('-'*50)

    cv = CountVectorizer()

    data = cv.fit_transform([c1, c2, c3])

    print(cv.get_feature_names_out())

    print(data.toarray())
    return None
hanzivec()

hanzivec
今天 很 残酷 ， 明天 更 残酷 ， 后天 很 美好 ， 但 绝对 大部分 是 死 在 明天 晚上 ， 所以 每个 人 不要 放弃 今天 。
--------------------------------------------------
我们 看到 的 从 很 远 星系 来 的 光是在 几百万年 之前 发出 的 ， 这样 当 我们 看到 宇宙 时 ， 我们 是 在 看 它 的 过去 。
--------------------------------------------------
如果 只用 一种 方式 了解 某样 事物 ， 你 就 不会 真正 了解 它 。 了解 事物 真正 含义 的 秘密 取决于 如何 将 其 与 我们 所 了解 的 事物 相 联系 。
--------------------------------------------------
['一种' '不会' '不要' '之前' '了解' '事物' '今天' '光是在' '几百万年' '发出' '取决于' '只用' '后天' '含义'
 '大部分' '如何' '如果' '宇宙' '我们' '所以' '放弃' '方式' '明天' '星系' '晚上' '某样' '残酷' '每个'
 '看到' '真正' '秘密' '绝对' '美好' '联系' '过去' '这样']
[[0 0 1 0 0 0 2 0 0 0 0 0 1 0 1 0 0 0 0 1 1 0 2 0 1 0 2 1 0 0 0 1 1 0 0 0]
 [0 0 0 1 0 0 0 1 1 1 0 0 0 0 0 0 0 1 3 0 0 0 0 1 0 0 0 0 2 0 0 0 0 0 1 1]
 [1 1 0 0 4 3 0 0 0 0 1 1 0 1 0 1 1 0 1 0 0 1 0 0 0 1 0 0 0 2 1 0 0 1 0 0]]


In [46]:
np.log10(1.5)

0.17609125905568124

In [47]:
# 规范{'l1'，'l2'}，默认='l2'
# 每个输出行都有单位范数，或者：
#
# 'l2'：向量元素的平方和为 1。当应用 l2 范数时，两个向量之间的余弦相似度是它们的点积。
#
# 'l1'：向量元素的绝对值之和为 1。参见preprocessing.normalize。


# 通过在文档频率上加一来平滑 idf 权重，就好像看到一个额外的文档包含集合中的每个术语恰好一次。防止零分裂。
# 比如训练集中有某个词，测试集中没有，就是生僻词，就会造成n(x)分母为零，log(n/n(x)),从而出现零分裂

def tfidfvec():
    """
    中文特征值化,计算tfidf值
    :return: None
    """
    c1, c2, c3 = cutword()

    print(c1, c2, c3)
    print('-'*50)
    print(type([c1, c2, c3]))
    print('-'*50)

    # smooth_idf=False,不平滑，就是不加1，True，平滑就是加1，防止零分裂，
    # 比如训练集中有某个词，测试集中没有，就是生僻词，就会造成n(x)分母为零，log(n/n(x)),从而出现零分裂
    tf = TfidfVectorizer(smooth_idf=True)

    data = tf.fit_transform([c1, c2, c3])

    print(tf.get_feature_names_out())
    print('-'*50)
    print(type(data))
    print('-'*50)
    print(data.toarray())

    return None


tfidfvec()

今天 很 残酷 ， 明天 更 残酷 ， 后天 很 美好 ， 但 绝对 大部分 是 死 在 明天 晚上 ， 所以 每个 人 不要 放弃 今天 。 我们 看到 的 从 很 远 星系 来 的 光是在 几百万年 之前 发出 的 ， 这样 当 我们 看到 宇宙 时 ， 我们 是 在 看 它 的 过去 。 如果 只用 一种 方式 了解 某样 事物 ， 你 就 不会 真正 了解 它 。 了解 事物 真正 含义 的 秘密 取决于 如何 将 其 与 我们 所 了解 的 事物 相 联系 。
--------------------------------------------------
<class 'list'>
--------------------------------------------------
['一种' '不会' '不要' '之前' '了解' '事物' '今天' '光是在' '几百万年' '发出' '取决于' '只用' '后天' '含义'
 '大部分' '如何' '如果' '宇宙' '我们' '所以' '放弃' '方式' '明天' '星系' '晚上' '某样' '残酷' '每个'
 '看到' '真正' '秘密' '绝对' '美好' '联系' '过去' '这样']
--------------------------------------------------
<class 'scipy.sparse._csr.csr_matrix'>
--------------------------------------------------
[[0.         0.         0.21821789 0.         0.         0.
  0.43643578 0.         0.         0.         0.         0.
  0.21821789 0.         0.21821789 0.         0.         0.
  0.         0.21821789 0.21821789 0.         0.43643578 0.
  0.21821789 0.         0.43643578 0.21821789 0.         0.
  0.         0.

# 4 特征处理，不同的特征拉到到同一个量纲

## 归一化
#### 归一化公式是：(X-Xmin)/(Xmax-Xmin)
#### 假设 nums1=[90, 2, 10, 40] Xmin=2 Xmax=90
#### (90-2)/(90-2)=88/88=1
#### (2-2)/(90-2)=0/88=0
#### (10-2)/(90-2)=8/88=0.09
#### (40-2)/(90-2)=38/88=0.43

In [82]:
def mm():
    """
    归一化处理
    :return: None
    """
    # 归一化缺点 容易受极值的影响
    #feature_range代表特征值范围，一般设置为(-1,1),默认是(0,1)
    mm = MinMaxScaler(feature_range=(-1,1))#这里是()元组
    nums1=[90, 2, 10, 40]
    nums2=[60, 4, 15, 45]
    nums3=[75, 3, 13, 46]
    #transform和fit_transform不同是，transform用于测试集，而且不会重新找最小值和最大值
    data= mm.fit_transform([nums1,nums2,nums3])
    print(data)



    return None


mm()

[[ 1.         -1.         -1.         -1.        ]
 [-1.          1.          1.          0.66666667]
 [ 0.          0.          0.2         1.        ]]


In [56]:
(np.square(1-2.333)+np.square(2-2.333)+np.square(4-2.333))/3
print('-'*50)
(1-2.333)/np.sqrt(1.55555)

--------------------------------------------------


-1.068779614944516

### 标准化
#### 标准化公式是：(X-Xmean)/S
#### S是标准差，Xmean是平均值
#### 标准差的公式是：S=sqrt((平方差和)/n)
#### 标准差是方差的算术平方根，标准差能反映一个数据集的离散程度
#### 假设 nums1=[90, 2, 10, 40] Xmean=35.5 S=35.5

In [ ]:
def stand():
    """
    标准化缩放，不是标准正太分布，只均值为0，方差为1的分布
    :return:
    """
    std = StandardScaler()

    data = std.fit_transform([[1., -1., 3.], [2., 4., 2.], [4., 6., -1.]])

    print(data)
    print('-' * 50)
    print(std.mean_)
    print('-' * 50)
    print(std.var_)
    print(std.n_samples_seen_)  # 样本数
    print('-' * 50)
    #为了证明上面输出的结果的均值是为0的，方差为1
    data1 = std.fit_transform([[-1.06904497, -1.35873244, 0.98058068],
                               [-0.26726124, 0.33968311, 0.39223227],
                               [1.33630621, 1.01904933, -1.37281295]])
    # print(data1)  #这个并不是我们想看的，没意义
    # 均值
    print(std.mean_)
    # 方差
    print(std.var_)
    # 样本数
    print(std.n_samples_seen_)
    return None


stand()

# 5 缺失值处理

In [78]:
#下面是填补，针对删除，可以用pd和np
def im():
    """
    缺失值处理
    :return:NOne
    """
    # NaN, nan,缺失值必须是这种格式,如果是np.nan,就不用管了
    # 如果是？号(或者其他符号)，就要replace换成np.nan
    # strategy='mean'是用均值填补，还有'median'中位数，'most_frequent'众数
    im = SimpleImputer(missing_values=np.nan, strategy='most_frequent')

    data = im.fit_transform([[3, 2], [np.nan, 3], [7,3], [4, 3]])

    print(data)

    return None


im()

[[3. 2.]
 [3. 3.]
 [7. 3.]
 [4. 3.]]


# 6 降维
#### 降维就是特征数变少
#### 降维可以提高模型训练速度（模型的参数减少了）

In [79]:
def var():
    """
    特征选择-删除低方差的特征
    :return: None
    """
    #默认只删除方差为0,threshold是方差阈值，删除比这个值小的那些特征
    var = VarianceThreshold(threshold=0.9)

    data = var.fit_transform([[0, 2, 0, 3],
                              [0, 1, 4, 3],
                              [0, 1, 1, 3]])

    print(data)
    # 获得剩余的特征的列编号
    print('The surport is %s' % var.get_support(True))
    return None


var()


[[0]
 [4]
 [1]]
The surport is [2]


In [80]:
def pca():
    """
    主成分分析进行特征降维
    :return: None
    """
    # n_ components:小数 0~1 90% 业界选择 90~95%
    #
    # 如果是整数   减少到的特征数量

    pca = PCA(n_components=0.9)

    data = pca.fit_transform([[2, 8, 4, 5], [6, 3, 0, 8], [5, 4, 9, 1]])

    print(data)

    return None


pca()



[[-3.13587302e-16  3.82970843e+00]
 [-5.74456265e+00 -1.91485422e+00]
 [ 5.74456265e+00 -1.91485422e+00]]
